In [1]:
import fitz
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from io import BytesIO
from PIL import Image
import os
from tqdm import tqdm
from time import time

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory as imreader
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, AveragePooling2D, InputLayer, Flatten, Resizing
from tensorflow.keras import losses, optimizers, metrics

In [2]:
train_ds = imreader("tf_model/data/train/",
                subset='training',
                validation_split=0.13,
                seed=123,
                color_mode="grayscale", 
                batch_size=8, 
                image_size=(20,20))

val_ds = imreader("tf_model/data/train/",
                subset='validation',
                validation_split=0.13,
                seed=123,
                color_mode="grayscale", 
                batch_size=8, 
                image_size=(20,20))
test_ds = imreader("tf_model/data/test//",
                color_mode="grayscale", 
                batch_size=8, 
                image_size=(20,20))

Found 570 files belonging to 10 classes.
Using 496 files for training.
Found 570 files belonging to 10 classes.
Using 74 files for validation.
Found 1505 files belonging to 10 classes.


In [3]:
conv2d = [64, 64]
kernelsize = (3,3)
poolsize = (2,2)
activation = "relu"

In [4]:
model = Sequential()
model.add(InputLayer((20,20,1)))

for f in conv2d:
    model.add(Conv2D(f, kernel_size=kernelsize, activation=activation))
    model.add(AveragePooling2D(pool_size=poolsize, padding="same"))

model.add(Flatten())
model.add(Dense(128, activation=activation))
model.add(Dense(10, activation="linear"))

In [5]:
model.compile(optimizer=optimizers.Adam(), 
              loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=metrics.SparseCategoricalAccuracy())

In [14]:
history = model.fit(train_ds, validation_data=val_ds, use_multiprocessing=True, epochs=1)

62/62 [==============================] - 0s 4ms/step - loss: 0.0092 - sparse_categorical_accuracy: 0.9980 - val_loss: 3.5680e-04 - val_sparse_categorical_accuracy: 1.0000


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 18, 18, 64)        640       
                                                                 
 average_pooling2d (Average  (None, 9, 9, 64)          0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 average_pooling2d_1 (Avera  (None, 4, 4, 64)          0         
 gePooling2D)                                                    
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 128)               1

In [15]:
model.evaluate(test_ds)

189/189 [==============================] - 0s 2ms/step - loss: 3.1101e-04 - sparse_categorical_accuracy: 1.0000


[0.00031100650085136294, 1.0]

In [20]:
model.save_weights("tf_model/models/conv_model/w")
model.save("tf_model/models/conv_model/model")

INFO:tensorflow:Assets written to: tf_model/models/conv_model/model\assets


INFO:tensorflow:Assets written to: tf_model/models/conv_model/model\assets


In [17]:
resizer = Resizing(20,20)

In [18]:
tests = []
for file in os.listdir("tf_model/data/numpy/"):
    test = np.load(f"tf_model/data/numpy/{file}")
    tests.append(test)
    test = np.expand_dims(test, axis=(0,-1))
    test = resizer(test)
    print(tf.argmax(tf.nn.softmax(model.predict(test)), axis=1))

1/1 [==============================] - 0s 12ms/step
tf.Tensor([3], shape=(1,), dtype=int64)
1/1 [==============================] - 0s 12ms/step
tf.Tensor([6], shape=(1,), dtype=int64)
1/1 [==============================] - 0s 11ms/step
tf.Tensor([4], shape=(1,), dtype=int64)
1/1 [==============================] - 0s 11ms/step
tf.Tensor([4], shape=(1,), dtype=int64)
1/1 [==============================] - 0s 12ms/step
tf.Tensor([4], shape=(1,), dtype=int64)


In [19]:
tests[1].shape

(29, 17)

In [13]:
img = resizer(np.expand_dims(img_to_array(load_img("tf_model/data/test/4/048_2.png", color_mode="grayscale")), axis=0))
tf.nn.softmax(model.predict(img))

1/1 [==============================] - 0s 12ms/step


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[4.0545544e-08, 2.4269549e-04, 5.6589092e-06, 2.6495873e-06,
        9.9929237e-01, 3.6653614e-04, 5.6155832e-06, 8.1094273e-05,
        9.3981140e-07, 2.3448245e-06]], dtype=float32)>